![Microsoft](https://raw.githubusercontent.com/microsoft/azuredatastudio/master/src/sql/media/microsoft-small-logo.png)
 
## Deploy SQL Server 2019 CTP 3.2 big data cluster on an existing Azure Kubernetes Service (AKS) cluster
 
This notebook walks through the process of deploying a <a href="https://docs.microsoft.com/sql/big-data-cluster/big-data-cluster-overview?view=sqlallproducts-allversions">SQL Server 2019 CTP 3.2 big data cluster</a> on an existing AKS cluster.
 
* Follow the instructions in the **Prerequisites** cell to install the tools if not already installed.
* Make sure you have the target cluster set as the current context in your kubectl config file.
        The config file would typically be under C:\Users\(userid)\.kube on Windows, and under ~/.kube/ for macOS and Linux for a default installation.
        In the kubectl config file, look for "current-context" and ensure it is set to the AKS cluster that the SQL Server 2019 CTP 3.2 big data cluster will be deployed to.
* The **Required information** cell will prompt you for password that will be used to access the cluster controller, SQL Server, and Knox.
* The values in the **Default settings** cell can be changed as appropriate.

### **Prerequisites** 
Ensure the following tools are installed and added to PATH before proceeding.
 
|Tools|Description|Installation|
|---|---|---|
|kubectl | Command-line tool for monitoring the underlying Kuberentes cluster | [Installation](https://kubernetes.io/docs/tasks/tools/install-kubectl/#install-kubectl-binary-using-native-package-management) |
|azdata | Command-line tool for installing and managing a big data cluster |[Installation](https://docs.microsoft.com/en-us/sql/big-data-cluster/deploy-install-azdata?view=sqlallproducts-allversions) |

### **Check dependencies**

In [1]:
import sys
def run_command():
    print("Executing: " + cmd)
    !{cmd}
    if _exit_code != 0:
        sys.exit(f'Command execution failed with exit code: {str(_exit_code)}.\n\t{cmd}\n')
    print(f'Successfully executed: {cmd}')

cmd = 'kubectl version --client=true'
run_command()
cmd = 'azdata --version'
run_command()

### **Show current context**

In [2]:
cmd = ' kubectl config current-context'
run_command()

### **Required information**

In [3]:
import getpass
mssql_password = getpass.getpass(prompt = 'SQL Server 2019 big data cluster controller password')
if mssql_password == "":
    sys.exit(f'Password is required')
confirm_password = getpass.getpass(prompt = 'Confirm password')
if mssql_password != confirm_password:
    sys.exit(f'Passwords do not match.')
print('Password accepted, you can also use the same password to access Knox and SQL Server.')

### **Default settings**

In [4]:
mssql_cluster_name = 'mssql-cluster'
mssql_controller_username = 'admin'
configuration_profile = 'aks-dev-test'
configuration_folder = 'mssql-bdc-configuration'
print(f'SQL Server big data cluster name: {mssql_cluster_name}')
print(f'SQL Server big data cluster controller user name: {mssql_controller_username}')
print(f'Deployment configuration profile: {configuration_profile}')
print(f'Deployment configuration: {configuration_folder}')

### **Create a deployment configuration file**

In [6]:
import os
os.environ["ACCEPT_EULA"] = 'yes'
cmd = f'azdata bdc config init --source {configuration_profile} --target {configuration_folder} --force'
run_command()
cmd = f'azdata bdc config replace -c {configuration_folder}/cluster.json -j metadata.name={mssql_cluster_name}'
run_command()

### **Create SQL Server 2019 big data cluster**

In [7]:
import os
print (f'Creating SQL Server 2019 big data cluster: {mssql_cluster_name} using configuration {configuration_folder}')
os.environ["CONTROLLER_USERNAME"] = mssql_controller_username
os.environ["CONTROLLER_PASSWORD"] = mssql_password
os.environ["MSSQL_SA_PASSWORD"] = mssql_password
os.environ["KNOX_PASSWORD"] = mssql_password
cmd = f'azdata bdc create -c {configuration_folder}'
run_command()

### **Login to SQL Server 2019 big data cluster**

In [8]:
cmd = f'azdata login --cluster-name {mssql_cluster_name}'
run_command()

### **Show SQL Server 2019 big data cluster endpoints**

In [9]:
import json,html,pandas
from IPython.display import *
pandas.set_option('display.max_colwidth', -1)
cmd = f'azdata bdc endpoint list'
cmdOutput = !{cmd}
endpoints = json.loads(''.join(cmdOutput))
endpointsDataFrame = pandas.DataFrame(endpoints)
endpointsDataFrame.columns = [' '.join(word[0].upper() + word[1:] for word in columnName.split()) for columnName in endpoints[0].keys()]
display(HTML(endpointsDataFrame.to_html(index=False, render_links=True)))

### **Connect to master SQL Server instance in Azure Data Studio**
Click the link below to connect to the master SQL Server instance of the SQL Server 2019 big data cluster.

In [10]:
sqlEndpoints = [x for x in endpoints if x['name'] == 'sql-server-master']
if sqlEndpoints and len(sqlEndpoints) == 1:
    connectionParameter = '{"serverName":"' + sqlEndpoints[0]['endpoint'] + '","providerName":"MSSQL","authenticationType":"SqlLogin","userName":"sa","password":' + json.dumps(mssql_password) + '}'
    display(HTML('<br/><a href="command:azdata.connect?' + html.escape(connectionParameter)+'"><font size="3">Click here to connect to master SQL Server instance</font></a><br/>'))
else:
    sys.exit('Could not find the master SQL Server instance endpoint')